# Random Forest on GPU with Rapids AI

In [ ]:
from vars import *

In [ ]:
if gpu == True:
    for pos in positions:    
        # Get user input for the week number to predict
        num_week = int(input("Enter the week to predict: "))

        import cudf
        import cuml
        import cupy as cp
        import pandas as pd
        import numpy as np
        from cuml.ensemble import RandomForestRegressor
        from cuml.model_selection import train_test_split, GridSearchCV
        from sklearn.impute import SimpleImputer

        # Load your dataset using cuDF
        data = cudf.read_csv("datasets/weekly_scoring.csv")

        # Preprocessing
        data = data[data['POS'] == pos]
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING LG', 'RECEIVING 20+', 'RECEIVING TD', 'RUSHING Y/A', 'RUSHING LG', 'RUSHING 20+', 'DATE', 'YEAR', 'WEIGHT'])
        # With the following lines to get dummies without drop_first:
        data = cudf.get_dummies(data, columns=['PLAYER'])
        
        columns_with_missing = data.to_pandas().isnull().any().index.tolist()
        
        # Convert problematic columns to cuPy as NumPy arrays
        data_numpy = data[columns_with_missing].to_pandas().values
        
        # Impute missing values with the mean of each column using NumPy
        imputer = SimpleImputer(strategy='mean')
        data_numpy = imputer.fit_transform(data_numpy)
        
        # Convert the imputed data back to a cuDF DataFrame
        data[columns_with_missing] = cudf.DataFrame(data_numpy, columns=columns_with_missing)


        # Define the list of variables to predict
        var_list = ['PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 'PASSING Y/A', 'PASSING TD', 'PASSING INT',
                    'PASSING SACKS', 'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK']

        # Separate the dataset into features (X) and the target variable (y)
        X = data.drop(var_list, axis=1)
        y = data['MISC FPTS']

        # Save a copy of the dataset
        df = data

        # Hyperparameter tuning using GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }

        # Convert X and y to cuPy arrays for consistency
        X = X.to_cupy()
        y = y.to_cupy()
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        
        grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        
        # Fit the model with the best hyperparameters
        best_rf_model = grid_search.best_estimator_
        best_rf_model.fit(X_train, y_train)


        # Get a list of unique player names after one-hot encoding
        unique_players = X.columns

        # Create a list of dictionaries to store the results
        results_list = []

        for player in unique_players:
            # Create a cuDF DataFrame with all zeros
            predict_data = cudf.DataFrame(np.zeros((1, len(X.columns)), dtype=np.float64))
            # Set the corresponding player's column to 1 for prediction
            predict_data[0, player] = 1
            predict_data = cp.array(predict_data)
            # Make a prediction for the player
            misc_fpts_prediction = best_rf_model.predict(predict_data)
            results_list.append({'Player': player, 'MISC FPTS': misc_fpts_prediction[0]})

        # Convert the list of dictionaries into a cuDF DataFrame
        results_df = cudf.DataFrame(results_list)

        # Save the results to a CSV file
        file_name = f"predictions/RFweek{num_week}{pos}.csv"
        results_df.to_pandas().to_csv(file_name, index=False)

        print(results_df.head(10))
